In [185]:
#BSM Calculations
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from math import log, sqrt, exp, pi, factorial as fac

In [178]:
#Important functions to conduct element-wise operations

#defining an elementwise log function
def elog(array):
    temp_array = []
    for item in array:
        temp_array.append(log(item))
    return np.asarray(temp_array)

#defining an elementwise exponentiation function
def eexp(array):
    temp_array = []
    for item in array:
        temp_array.append(exp(item))
    return np.asarray(temp_array)

#defining an elementwise factorial function
def efac(array):
    temp_array = []
    for item in array:
        temp_array.append(fac(item))
    return np.asarray(temp_array)

In [233]:
#Merton Option Model

#Parameter values"
tau = 1/12        #maturity of option
logr1 = tau*0.020
q1 = exp(-logr1)
ep = 0.0400
sigmare = 0.1800

#True distribution (sigma is all we need here)
omega = 1.5120
theta = -0.0259
delta = 0.0407
mu = ep - omega*theta
sigma = sqrt(sigmare**2-omega*(theta**2+delta**2))

#Risk-neutral distribution
omegas = 1.5120
deltas = 0.0981
thetas = log(1-0.0482) - deltas**2/2

#set mu* to satisfy arb condition
mus = -sigma**2/2 - omegas*(exp(thetas+deltas**2/2)-1)

#2. Option Prices (Merton model)

#Strike grid
ngrid = 10
bscale = 0.08 #bscale = 0.3
bgrid = bscale*np.arange(-ngrid,ngrid+1,1)/ngrid + 1
logbgrid = elog(bgrid)

#probs
nprob = 10
jgrid = np.arange(0,nprob+1)
probs = exp(-tau*omegas)*(tau*omegas)**jgrid/efac(jgrid)
checksumprobs = sum(probs)

#vectorize instead of looping
logbarray = np.transpose(logbgrid*np.ones((nprob+1,len(logbgrid))))

#base parameters
kappa1j = tau*mus + jgrid*thetas
kappa1jarray = kappa1j*np.ones((2*ngrid+1,1))
kappa2j = tau*sigma**2+jgrid*delta**2 #does not equal the code output from Matlab for some reason. Syntax is correct though
kappa2jarray = kappa2j*np.ones((2*ngrid+1,1))
darray = (logbarray - logr1 - kappa1jarray)/np.sqrt(kappa2jarray)
putarray = q1*np.exp(logbarray)*norm.cdf(darray) - np.exp(kappa1jarray+kappa2jarray/2)*norm.cdf(darray-np.sqrt(kappa2jarray))
put_ase = putarray*probs

In [232]:
kappa2j

array([ 0.00240676,  0.00406325,  0.00571974,  0.00737623,  0.00903272,
        0.01068921,  0.0123457 ,  0.01400219,  0.01565868,  0.01731517,
        0.01897166])